<a href="https://colab.research.google.com/github/Henry-0810/Artificial-Intelligence/blob/main/chinese_chess_search_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Search Tree Algorithm Project
## End Game Steps of Chinese Chess (Xiang Qi)
Since full-game Chinese Chess is too complex, I plan to only focus on specific endgame scenarios.
1. King + Chariot vs. King (Basic but useful)
2. King + Cannon vs. King + Soldier (Intermediate)
3. King + Horse vs. King (More complex, requires mobility evaluation)

These scenarios are chosen because:
- The search tree remains manageable.
- The AI can calculate winning or drawing strategies.
- It demonstrates Minimax’s effectiveness.

---

**Some extra information about chinese chess:**
1. Chariot only can move up, down, left and right. It can capture any chess pieces, acts like a Rook in classic chess.
2. Cannon only can move up, down, left and right. A cannon must jump over a chess piece in its path to capture opponent's chess piece.
3. Soldier can only move one step forward, but once it moves pass the river, which is the mid line of the chess board, it can then move one step left, right and forward.
4. The Horse moves one point horizontally or vertically, and then one point diagonally. It cannot move in a direction where there is a piece blocking it along the path of movement.

---

**References:**
- [XiangQi Guide](https://www.xiangqi.com/how-to-play-xiangqi)

### **Implementation**

In [25]:
from copy import deepcopy

Scenario Loader


In [26]:
def load_scenario(index):
    board = [['.' for _ in range(9)] for _ in range(10)]

    if index == 1:
        board[0][4] = 'BK'
        board[9][4] = 'RK'
        board[7][4] = 'RR'  # Red Rook

    elif index == 2:
        board[0][4] = 'BK'
        board[1][4] = 'BS'
        board[9][4] = 'RK'
        board[7][4] = 'RC'  # Red Cannon

    elif index == 3:
        board[0][4] = 'BK'
        board[9][4] = 'RK'
        board[7][3] = 'RH'  # Red Horse

    return board

In [27]:
class CurrentBoard:
    def __init__(self, board_state=None):
        if board_state:
            self.board = deepcopy(board_state)
        else:
            self.board = load_scenario(1)
        self.state = 'U'

    def display(self):
        for row in self.board:
            print(" ".join(row))
        print("\n")

    def other(self, piece):
        return 'B' if piece == 'R' else 'R'

    def get_piece_owner(self, piece):
        return piece[0] if piece != '.' else None

    def state_of_board(self):
        red_king = black_king = False
        for row in self.board:
            for cell in row:
                if cell == 'RK':
                    red_king = True
                if cell == 'BK':
                    black_king = True
        if not red_king:
            return 'B_WIN'
        if not black_king:
            return 'R_WIN'
        if len(self.all_possible_moves('R')) == 0 and len(self.all_possible_moves('B')) == 0:
            return 'D'
        return 'U'

    def in_bounds(self, r, c):
        return 0 <= r < 10 and 0 <= c < 9

    def in_palace(self, r, c, owner):
        if owner == 'B':
            return 0 <= r <= 2 and 3 <= c <= 5
        elif owner == 'R':
            return 7 <= r <= 9 and 3 <= c <= 5
        return False

    def all_possible_moves(self, player):
        moves = []
        for r in range(10):
            for c in range(9):
                piece = self.board[r][c]
                if piece != '.' and self.get_piece_owner(piece) == player:
                    moves.extend(self.get_moves_for_piece(piece, r, c))
        return moves

    def get_moves_for_piece(self, piece, r, c):
        moves = []
        owner = self.get_piece_owner(piece)

        def add_move(nr, nc):
            if self.in_bounds(nr, nc):
                if piece[1] == 'K' and not self.in_palace(nr, nc, owner):
                    return
                target = self.board[nr][nc]
                if target == '.' or self.get_piece_owner(target) != owner:
                    new_board = deepcopy(self.board)
                    new_board[nr][nc] = piece
                    new_board[r][c] = '.'
                    moves.append(CurrentBoard(new_board))

        if piece[1] == 'K':  # King
            for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
                add_move(r+dr, c+dc)

        elif piece[1] == 'R':  # Rook (Chariot)
            for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
                nr, nc = r + dr, c + dc
                while self.in_bounds(nr, nc):
                    if self.board[nr][nc] == '.':
                        add_move(nr, nc)
                    else:
                        if self.get_piece_owner(self.board[nr][nc]) != owner:
                            add_move(nr, nc)
                        break
                    nr += dr
                    nc += dc

        elif piece[1] == 'C':  # Cannon
            for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
                nr, nc = r + dr, c + dc
                jumped = False
                while self.in_bounds(nr, nc):
                    if not jumped:
                        if self.board[nr][nc] == '.':
                            add_move(nr, nc)
                        else:
                            jumped = True
                    else:
                        if self.board[nr][nc] != '.' and self.get_piece_owner(self.board[nr][nc]) != owner:
                            add_move(nr, nc)
                            break
                        elif self.board[nr][nc] != '.':
                            break
                    nr += dr
                    nc += dc

        elif piece[1] == 'H':  # Horse
            horse_moves = [(-2,-1), (-2,1), (2,-1), (2,1),
                           (-1,-2), (-1,2), (1,-2), (1,2)]
            legs = [(-1,0), (-1,0), (1,0), (1,0),
                    (0,-1), (0,-1), (0,1), (0,1)]
            for i, (dr, dc) in enumerate(horse_moves):
                leg_r, leg_c = r + legs[i][0], c + legs[i][1]
                if not self.in_bounds(leg_r, leg_c) or self.board[leg_r][leg_c] != '.':
                    continue
                add_move(r + dr, c + dc)

        elif piece[1] == 'S':  # Soldier
            dr = -1 if owner == 'B' else 1
            add_move(r + dr, c)
            if (owner == 'B' and r <= 4) or (owner == 'R' and r >= 5):
                add_move(r, c - 1)
                add_move(r, c + 1)

        return moves


In [28]:
class SearchTreeNode:
    def __init__(self, board_instance, playing_as, ply=0):
        self.children = []
        self.value_is_assigned = False
        self.ply_depth = ply
        self.current_board = board_instance
        self.move_for = playing_as

        MAX_PLY_DEPTH = 4
        board_state = board_instance.state_of_board()
        if board_state == 'U' and ply < MAX_PLY_DEPTH:
            self.generate_children()
        else:
            self.value = self.evaluate_terminal_state(board_state)
            self.value_is_assigned = True

    def evaluate_terminal_state(self, board_state):
        if board_state == 'D':
            return 0
        elif board_state == f'{self.move_for}_WIN':
            return 1
        else:
            return -1


    def min_max_value(self):
        if self.value_is_assigned:
            return self.value
        child_values = [child.min_max_value() for child in self.children]
        if (self.ply_depth % 2) == 0:
            self.value = max(child_values)
        else:
            self.value = min(child_values)
        self.value_is_assigned = True
        return self.value

    def generate_children(self):
        for next_board in self.current_board.all_possible_moves(self.move_for):
            self.children.append(SearchTreeNode(next_board, self.current_board.other(self.move_for), self.ply_depth + 1))


In [29]:
def play_xiangqi_endgame():
    scenario = int(input("Choose scenario (1 = Rook, 2 = Cannon+Soldier, 3 = Horse): "))
    cb = CurrentBoard(load_scenario(scenario))
    player = 'R'
    turn = 0

    while cb.state == 'U':
        print(f"Turn {turn}, {player}'s move")
        cb.display()

        tree = SearchTreeNode(cb, player)
        tree.min_max_value()
        best_move = max(tree.children, key=lambda x: x.value)
        cb = best_move.current_board

        player = cb.other(player)
        turn += 1

    print(f"Game Over! Result: {cb.state}")
    cb.display()

In [30]:
play_xiangqi_endgame()

Choose scenario (1 = Rook, 2 = Cannon+Soldier, 3 = Horse): 1
Turn 0, R's move
. . . . BK . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . RR . . . .
. . . . . . . . .
. . . . RK . . . .


Turn 1, B's move
. . . . BK . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . RR . . . .
. . . . . . . . .
. . . . . . . . .
. . . . RK . . . .


Turn 2, R's move
. . . . . . . . .
. . . . BK . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . RR . . . .
. . . . . . . . .
. . . . . . . . .
. . . . RK . . . .


Turn 3, B's move
. . . . . . . . .
. . . . BK . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . RR . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . RK . . . .




KeyboardInterrupt: 